## `Model_1`
### Target:
- Getting the skeleton right
- Perform MaxPooling after the layer with 5 rececptive field
- Include dropout of 10% after all convolutional layers
- Have fully convolutional layer instead of fully connected layer

### Results:
- Parameters: 8.2 K
- Best Train Accuracy: 98.07
- Best Test Accuracy: 98.69

### Analysis:
- Model is underfitting - Current learning rate 0.001 might be too slow; LR scheduler can help
- Adam might perform better than SGD optimizer 
- May be add more channels in initial convolutional layers

# Import Libraries

In [2]:
! nvidia-smi

zsh:1: command not found: nvidia-smi


In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Data Transformations

We first start with defining our data transformations. We need to think what our data is and how can we augment it to correct represent images which it might not see otherwise.


In [4]:
# Train Phase transformations
train_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

# Test Phase transformations
test_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])


# Dataset and Creating Train/Test Split

In [5]:
from utils import prepare_mnist_data
train_loader, test_loader = prepare_mnist_data(
    train_transforms, test_transforms, batch_size=256)

CUDA Available? False


# The model

# Model Params
Can't emphasize on how important viewing Model Summary is.
Unfortunately, there is no in-built model visualizer, so we have to take external help

In [6]:
!pip install torchsummary
from torchsummary import summary
from model import Net
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net(
    list('CCcPCCCCGc'),
    [16, 16, 10, 10,
     16, 8, 8, 16,
     16, 10],
    dropout_value=0.03).to(device)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           2,304
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
           Dropout-8           [-1, 16, 28, 28]               0
            Conv2d-9           [-1, 10, 28, 28]             160
        MaxPool2d-10           [-1, 10, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           1,440
             ReLU-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
          Dropout-14           [-1,

# Training and Testing

Looking at logs can be boring, so we'll introduce **tqdm** progressbar to get cooler logs.

Let's write train and test functions

# Let's Train and test our model

In [14]:
from model import train, test
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=1.36038076877594 Batch_id=468 Accuracy=39.93: 100%|██████████| 469/469 [00:18<00:00, 24.99it/s]



Test set: Average loss: 1.3963, Accuracy: 5309/10000 (53.09%)

EPOCH: 1


Loss=0.6103813052177429 Batch_id=468 Accuracy=76.34: 100%|██████████| 469/469 [00:19<00:00, 23.56it/s]



Test set: Average loss: 0.6982, Accuracy: 8568/10000 (85.68%)

EPOCH: 2


...


Test set: Average loss: 0.0469, Accuracy: 9870/10000 (98.70%)

EPOCH: 18


Loss=0.04437898471951485 Batch_id=468 Accuracy=98.06: 100%|██████████| 469/469 [00:17<00:00, 26.70it/s]



Test set: Average loss: 0.0435, Accuracy: 9878/10000 (98.78%)

EPOCH: 19


Loss=0.05325952544808388 Batch_id=468 Accuracy=98.07: 100%|██████████| 469/469 [00:18<00:00, 25.20it/s]



Test set: Average loss: 0.0447, Accuracy: 9869/10000 (98.69%)

